In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import chi2_contingency, mannwhitneyu, ttest_ind, pearsonr, spearmanr, shapiro, kruskal, f_oneway

In [5]:
df = pd.read_csv('corrected_data.csv', skipinitialspace=True, sep=',', index_col=0)
df.head()

,год,время_года,климат,город,страна,способ_охлаждения,режим_при_смешанном_типе_охлаждения,способ_обогрева,возраст,пол,...,скорость_воздуха,рост,вес,занавески,вентилятор,окно,двери,отопление,среднемесячная_температура_на_улице,количество_рекламаций
0,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,22,Мужской,...,0.26,170.2,63.0,1.0,1.0,NaN,NaN,NaN,28.9,0
1,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,40,Мужской,...,0.12,170.2,63.0,0.0,0.0,NaN,NaN,NaN,84.0,0
2,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,42,Мужской,...,0.08,170.2,63.0,0.0,0.0,NaN,NaN,NaN,32.8,0
3,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,27,Мужской,...,27.16,170.2,63.0,1.0,1.0,NaN,NaN,NaN,28.9,1
4,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,46,Мужской,...,63.83,170.2,63.0,1.0,1.0,NaN,NaN,NaN,32.8,0


In [6]:
def hi2(g1, g2):
    ct = pd.crosstab(g1, g2)
    chi2 = chi2_contingency(ct)
    return (chi2.statistic / (df.shape[0] * (min(df.shape) - 1))) ** 0.5, chi2.pvalue

def corr(g1, g2, type1, type2):
    res = 0
    if type1 != type2:
        norm = 0
        if type1 == 'cat':
            norm = shapiro(g2).pvalue >= 0.05
            gs = [g2[g1 == i] for i in g1.unique()]
            ln = len(g1.unique())
        else:
            gs = [g1[g2 == i] for i in g2.unique()]
            norm = shapiro(g1).pvalue >= 0.05
            ln = len(g2.unique())
        if not norm:
            if ln == 2:
                res = mannwhitneyu(g1, g2)
                print('Method: Mann-Whitneyu')
            else:
                res = kruskal(*gs)
                print('Method: Kruskal-Wallis')
        else: 
            if ln == 2:
                res = ttest_ind(g1, g2)
                print('Method: Student')
            else:
                print(gs)
                res = f_oneway(*gs)
                print('Method: ANOVA')
    else:
        if type1 == 'cat':
            print('Method: Chi2')
            return hi2(g1, g2)
        else:
            norm = shapiro(g1).pvalue >= 0.05 and shapiro(g2).pvalue >= 0.05
            if not norm:
                res = spearmanr(g1, g2)
                print('Method: Spearman')
            else:
                res = pearsonr(g1, g2)
                print('Method: Pearson')
    return res[0], res[1]

In [7]:
corr(df['способ_охлаждения'], df['оценка_комфорта'], 'cat', 'cat')

Method: Chi2


(0.04647619042319019, 0.3481673969162561)

Способ охлаждения не вляет на оценку комфорта. Значение корреляции маленькое, следовательно, зависимость очень слабая. Уровень доверия больше 0.05, значит, ответу нельзя доверять.

In [8]:
corr(df['пол'], df['оценка_комфорта'], 'cat', 'cat')

Method: Chi2


(0.0251014068577231, 0.8409239292046167)

Здесь уровень корреляции также маленький, однако p-уровень больше 0.05, поэтому по имеющимся данным выводов мы делать не можем

In [9]:
def changeage(x):
    if x <= 44:
        return 'молодой возраст'
    if 45 <= x <= 59:
        return 'средний возраст'
    if x >= 60:
        return 'пожилой возраст'
df['возрастная_категория'] = df['возраст'].map(changeage)

In [10]:
corr(df['возрастная_категория'], df['оценка_комфорта'], 'cat', 'cat')

Method: Chi2


(0.046012383720655976, 0.32870686581672814)

Здесь уровень корреляции также маленький, однако p-уровень больше 0.05, поэтому по имеющимся данным выводов мы делать не можем

In [11]:
corr(df['количество_рекламаций'], df['оценка_комфорта'], 'num', 'cat')

Method: Kruskal-Wallis


(nan, nan)

Зависимость сильная, при этом p-уровень очень маленький, поэтому можно сказать, что между количеством рекламаций и оценкой комфорта есть связь 

In [82]:
corr(df['страна'], df['оценка_комфорта'], 'cat', 'num') 

Method: Kruskal-Wallis


(57.44327573695403, 3.360091952598967e-13)

Зависимость сильная, при этом p-уровень очень маленький, поэтому можно сказать, что между страной и оценкой комфорта есть связь 